In [ ]:
!pip install --upgrade langchain google-generativeai yfinance


In [ ]:
import os
import google.generativeai as genai
import yfinance as yf
import pandas as pd
from typing import Optional, List, Mapping, Any
from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate

# Replace below with your Gemini API key
os.environ["GOOGLE_API_KEY"] = "paste your key here"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [ ]:
def fetch_stock_history(ticker: str) -> pd.DataFrame:
    # Add .NS suffix for NSE stocks automatically if needed
    if not ticker.endswith(".NS") and ticker.isalpha():
        ticker = ticker + ".NS"
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max")
    if hist.empty:
        raise ValueError(f"No data found for ticker {ticker}")
    return hist

def summarize_price_history(hist_df: pd.DataFrame) -> str:
    summary = f"Data from {hist_df.index[0].date()} to {hist_df.index[-1].date()}.\n"
    summary += f"Total days: {len(hist_df)}\n"
    summary += f"Opening price range: {hist_df['Open'].min():.2f} to {hist_df['Open'].max():.2f}\n"
    summary += f"Closing price range: {hist_df['Close'].min():.2f} to {hist_df['Close'].max():.2f}\n"
    summary += f"Average volume: {hist_df['Volume'].mean():,.0f}\n"
    summary += f"Recent closing prices (last 5 days): {hist_df['Close'][-5:].to_list()}\n"
    return summary

def fetch_stock_news_simulated(ticker: str) -> str:
    articles = [
        f"Article 1: {ticker} recently announced a new product line which analysts say could boost revenue.",
        f"Article 2: {ticker} has faced supply chain issues impacting Q2 earnings.",
        f"Article 3: {ticker} CEO hinted at potential market expansion in Asia next year.",
        f"Article 4: {ticker} stock price showed unusual volume spikes last month.",
        f"Article 5: Insider trading activities in {ticker} have been unusually high.",
    ]
    return "\n".join(articles)


In [ ]:
prompt_template = """
You are a financial expert AI assistant analyzing stock data.

Stock Ticker: {ticker}

Stock Price History Summary:
{price_summary}

Additional Info / Articles:
{articles}

Based on the above, provide:
1. A detailed analysis of the stock's historical performance.
2. Some investment tips and recommendations.
3. As many hidden or less obvious insights from the articles and data as possible that typical investors might miss (aim for at least 10 if the information allows).

Answer in a clear, concise way.
"""

prompt = PromptTemplate(
    input_variables=["ticker", "price_summary", "articles"],
    template=prompt_template
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import os
import yfinance as yf
import pandas as pd
from typing import Optional, List, Mapping, Any
from langchain.prompts import PromptTemplate

# Get the API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Initialize the Gemini 1.5 Flash model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")


def fetch_stock_history(ticker: str) -> pd.DataFrame:
    # Add .NS suffix for NSE stocks automatically if needed
    if not ticker.endswith(".NS") and ticker.isalpha():
        ticker = ticker + ".NS"
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max")
    if hist.empty:
        raise ValueError(f"No data found for ticker {ticker}")
    return hist

def summarize_price_history(hist_df: pd.DataFrame) -> str:
    summary = f"Data from {hist_df.index[0].date()} to {hist_df.index[-1].date()}.\n"
    summary += f"Total days: {len(hist_df)}\n"
    summary += f"Opening price range: {hist_df['Open'].min():.2f} to {hist_df['Open'].max():.2f}\n"
    summary += f"Closing price range: {hist_df['Close'].min():.2f} to {hist_df['Close'].max():.2f}\n"
    summary += f"Average volume: {hist_df['Volume'].mean():,.0f}\n"
    summary += f"Recent closing prices (last 5 days): {hist_df['Close'][-5:].to_list()}\n"
    return summary

def fetch_stock_news_simulated(ticker: str) -> str:
    articles = [
        f"Article 1: {ticker} recently announced a new product line which analysts say could boost revenue.",
        f"Article 2: {ticker} has faced supply chain issues impacting Q2 earnings.",
        f"Article 3: {ticker} CEO hinted at potential market expansion in Asia next year.",
        f"Article 4: {ticker} stock price showed unusual volume spikes last month.",
        f"Article 5: Insider trading activities in {ticker} have been unusually high.",
    ]
    return "\n".join(articles)

def analyze_stock(ticker: str, llm: ChatGoogleGenerativeAI) -> str:
    try:
        hist = fetch_stock_history(ticker)
        price_summary = summarize_price_history(hist)
        articles = fetch_stock_news_simulated(ticker)

        formatted_prompt = prompt.format(
            ticker=ticker,
            price_summary=price_summary,
            articles=articles
        )

        response = llm.invoke(formatted_prompt)
        return response.content
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
import re

ticker_input = input("Enter Stock Ticker Symbol (e.g., AAPL or HDFCBANK): ").strip().upper()
result = analyze_stock(ticker_input, llm)

print("\n=== Stock Analysis ===\n")

# Use regex to split on periods only when followed by space and uppercase letter
sentences = re.split(r'\.\s+(?=[A-Z])', result)

for sentence in sentences:
    sentence = sentence.strip()
    if sentence:
        print(sentence + ".")


Enter Stock Ticker Symbol (e.g., AAPL or HDFCBANK): RELIANCE

=== Stock Analysis ===

## RELIANCE Stock Analysis:

**1.
Historical Performance Analysis:**

RELIANCE has shown substantial growth over its history (1996-2025), with a closing price increase from ₹3.46 to ₹1595.48.
This represents a massive appreciation, though the exact CAGR (Compound Annual Growth Rate) requires more detailed data.
The average daily volume of 58,686,681 indicates significant trading activity.
However, the recent closing prices show a slight fluctuation within a narrow range (₹1367.80 - ₹1392.80), suggesting potential consolidation or indecision in the market.
The wide range of opening and closing prices throughout its history highlights the volatility inherent in the stock.

**2.
Investment Tips and Recommendations:**

* **Long-term outlook:**  Given the historical growth and potential future catalysts (new product line, market expansion), a long-term investment strategy might be suitable for risk-toleran